# Transformer-motion2label2

In [2]:
%install-location /notebooks/language2motion.gt/swift-install
%install-swiftpm-flags -c release
%install '.package(path: "/notebooks/language2motion.gt/code")' Datasets MotionModels ImageClassificationModels TextModels ModelSupport SummaryWriter

Installing packages:
	.package(path: "/notebooks/language2motion.gt/code")
		Datasets
		MotionModels
		ImageClassificationModels
		TextModels
		ModelSupport
		SummaryWriter
With SwiftPM flags: ['-c', 'release']
Working in: /tmp/tmpvnybirfl/swift-install
[1/2] Compiling Datasets ArrayUtils.swift
[2/3] Compiling TextModels Attention.swift
[3/4] Compiling MotionModels DenseMotionClassifier.swift
[4/5] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
[5/5] Linking libjupyterInstalledPackages.so
Initializing Swift...
Installation complete!


In [3]:
import Foundation
import TensorFlow
import Datasets
import MotionModels
import ImageClassificationModels
import TextModels
import ModelSupport
import SummaryWriter

In [4]:
import PythonKit

let metrics = Python.import("sklearn.metrics")

In [5]:
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

('inline', 'module://ipykernel.pylab.backend_inline')


# load dataset

In [6]:
let batchSize = 4
let maxSequenceLength =  1000
let runName = "run_9"
let balanceClassSamples: Int? = 600

print("batchSize: \(batchSize)")
print("maxSequenceLength: \(maxSequenceLength)")
print("runName: \(runName)")

// let serializedDatasetURL = URL(fileURLWithPath: "/notebooks/language2motion.gt/data/motion_dataset_v2.normalized.plist")
// let serializedDatasetURL = URL(fileURLWithPath: "/notebooks/language2motion.gt/data/motion_dataset.motion_flag.normalized.sampled.500.plist")
let serializedDatasetURL = URL(fileURLWithPath: "/notebooks/language2motion.gt/data/motion_dataset.motion_flag.normalized.plist")
let labelsURL = URL(fileURLWithPath: "/notebooks/language2motion.gt/data/labels_ds_v2.csv")

print("\nLoading dataset...")
let dataset = try! Motion2Label(
    serializedDatasetURL: serializedDatasetURL,
    labelsURL: labelsURL,
    maxSequenceLength: maxSequenceLength,
    batchSize: batchSize,
    balanceClassSamples: balanceClassSamples
) { 
    // TODO: move this to dataset class
    (example: Motion2LabelExample) -> LabeledMotionBatch in
    let motionFrames = Tensor<Float>(example.motionSample.motionFramesArray)
    let motionFlag = Tensor<Int32>(motionFrames[0..., 44...44].squeezingShape(at: 1))
    let origMotionFramesCount = Tensor<Int32>(Int32(motionFrames.shape[0]))
    let motionBatch = MotionBatch(motionFrames: motionFrames, motionFlag: motionFlag, origMotionFramesCount: origMotionFramesCount)
    let label = Tensor<Int32>(Int32(example.label!.idx))
    return LabeledMotionBatch(data: motionBatch, label: label)
}

print("dataset.trainingExamples.count: \(dataset.trainingExamples.count)")
print("dataset.validationExamples.count: \(dataset.validationExamples.count)")

batchSize: 4
maxSequenceLength: 1000
runName: run_9

Loading dataset...
MotionDataset(motionSamples: 3911)
Class balancing...
(1216, 480, 120)
(644, 480, 120)
(103, 480, 21)
(400, 480, 80)
(649, 480, 120)
dataset.trainingExamples.count: 2400
dataset.validationExamples.count: 461


# debug batch related crash

In [9]:
dataset.validationExamples.count

461


In [11]:
print(type(of:dataset.validationExamples[0]))

(data: MotionBatch, label: Tensor<Int32>)


In [13]:
dataset.validationExamples[0].data.motionFrames.shape

▿ [543, 45]
  ▿ dimensions : 2 elements
    - 0 : 543
    - 1 : 45


In [14]:
dataset.validationExamples[460].data.motionFrames.shape

▿ [801, 45]
  ▿ dimensions : 2 elements
    - 0 : 801
    - 1 : 45


In [8]:
dataset.validationBatches.count

116


In [19]:
for (idx, batch) in dataset.validationBatches.enumerated() {
    let valBatchSize = batch.data.motionFrames.shape[0]
    let (documents, labels) = (batch.data, Tensor<Int32>(batch.label))
    if documents.motionFrames.shape.count < 3 || idx==114 {
        print("batch \(idx)")
        print("documents.motionFrames.shape: \(documents.motionFrames.shape)")
//         let numFeatures = documents.motionFrames.shape[2]
    }
}

batch 114
documents.motionFrames.shape: [4, 1000, 45]
batch 115
documents.motionFrames.shape: [801, 45]


In [35]:
4*114+1

457


In [83]:
let examples = dataset.validationExamples[4*114-1..<460]
examples.count

5


In [84]:
let batches1 = Array(examples.inBatches(of: batchSize))
batches1.count

2


In [89]:
extension Tensor where Scalar: Numeric {
    public func paddedOrCropped(to width: Int) -> Tensor<Scalar> {
        // pads or crops one- or two-dimensional tensor along 0-th axis
        let rank = self.shape.count
        let currentWidth = self.shape[0]
        let paddingSize = Swift.max(width - currentWidth, 0)
        let maxCropping = Swift.max(currentWidth - width, 0)
        let nCropping = (maxCropping>0) ? Int.random(in: 0 ..< maxCropping) : 0
        var sizes: [(before: Int, after: Int)] = [(before: 0, after: paddingSize)]
        if rank > 1 {
            sizes.append((before: 0, after: 0))
        }
        return self[nCropping..<nCropping+width].padded(forSizes: sizes)
    }
}

In [121]:
extension MotionBatch: Collatable {
  /// Creates an instance from collating `samples`.
  public init<BatchSamples: Collection>(collating samples: BatchSamples)
  where BatchSamples.Element == Self {
      if samples.count == 1 {
          print(1)
      }
    self.init(
      motionFrames: .init(concatenating: samples.map({$0.motionFrames.expandingShape(at: 0)})), 
      motionFlag: .init(concatenating: samples.map({$0.motionFlag.expandingShape(at: 0)})),
      origMotionFramesCount: .init(concatenating: samples.map({$0.origMotionFramesCount.expandingShape(at: 0)}))
    )
  }
}

In [122]:
extension Collection where Element == MotionBatch {
  /// Returns the elements of `self`, padded to `maxLength` if specified
  /// or the maximum length of the elements in `self` otherwise.
  public func paddedAndCollated2(to maxLength: Int? = nil) -> MotionBatch {
    let maxLength = maxLength ?? self.map { $0.motionFrames.shape[1] }.max()!
    let paddedMotions = self.map { example -> MotionBatch in
        return MotionBatch(
        motionFrames: example.motionFrames.paddedOrCropped(to: maxLength),
        motionFlag: example.motionFlag.paddedOrCropped(to: maxLength),
        origMotionFramesCount: example.origMotionFramesCount)
    }
    if count == 1 { 
        let pm = paddedMotions[0]
        return MotionBatch(
            motionFrames: pm.motionFrames.expandingShape(at: 0), 
            motionFlag: pm.motionFlag.expandingShape(at: 0), 
            origMotionFramesCount: pm.origMotionFramesCount.expandingShape(at: 0))
    } else {
        return paddedMotions.collated
    }
  }
}

In [123]:
for labeledMotionBatchSequence in batches1 {
    print("labeledMotionBatchSequence: \(type(of:labeledMotionBatchSequence))")
    for labeledMotionBatch: LabeledMotionBatch in labeledMotionBatchSequence {
        print("labeledMotionBatch: \(type(of:labeledMotionBatch))")
        print(labeledMotionBatch.data.motionFrames.shape)
    }
    let motionBatch: MotionBatch = labeledMotionBatchSequence.map(\.data).paddedAndCollated2(to: maxSequenceLength)
    print("motionBatch: \(type(of:motionBatch))")
    print("motionFrames:", motionBatch.motionFrames.shape)
    print("motionFlag:", motionBatch.motionFlag.shape)
    print("origMotionFramesCount:", motionBatch.origMotionFramesCount.shape)
}

labeledMotionBatchSequence: LazyMapSequence<ArraySlice<Motion2LabelExample>, (data: MotionBatch, label: Tensor<Int32>)>
labeledMotionBatch: (data: MotionBatch, label: Tensor<Int32>)
[667, 45]
labeledMotionBatch: (data: MotionBatch, label: Tensor<Int32>)
[590, 45]
labeledMotionBatch: (data: MotionBatch, label: Tensor<Int32>)
[540, 45]
labeledMotionBatch: (data: MotionBatch, label: Tensor<Int32>)
[926, 45]
motionBatch: MotionBatch
motionFrames: [4, 1000, 45]
motionFlag: [4, 1000]
origMotionFramesCount: [4]
labeledMotionBatchSequence: LazyMapSequence<ArraySlice<Motion2LabelExample>, (data: MotionBatch, label: Tensor<Int32>)>
labeledMotionBatch: (data: MotionBatch, label: Tensor<Int32>)
[391, 45]
motionBatch: MotionBatch
motionFrames: [1, 1000, 45]
motionFlag: [1, 1000]
origMotionFramesCount: [1]


In [56]:
batches1[0].data

: 

In [50]:
let batches2 = batches1.map { 
     LabeledMotionBatch(
        data: $0.map(\.data).paddedAndCollated(to: maxSequenceLength),
        label: Tensor($0.map(\.label))
    )
}
batches2.count

2


In [53]:
batches2[0].data.motionFrames.shape

▿ [4, 1000, 45]
  ▿ dimensions : 3 elements
    - 0 : 4
    - 1 : 1000
    - 2 : 45


In [54]:
batches2[1].data.motionFrames.shape

▿ [391, 45]
  ▿ dimensions : 2 elements
    - 0 : 391
    - 1 : 45
